In [ ]:
import pandas as pd
from pandas.plotting import table 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
commands = []
with open("table.txt") as file:
    for line in file:
        commands.append(line.strip())

In [ ]:
def findOperation(line):
    if line.find("is-a")!=-1:
        return ("is-a")
    elif line.find("has-row")!=-1:
        return ("has-row")
    elif line.find("has-slot")!=-1:
        return ("has-slot")
    elif line.find("starts-at")!=-1:
        return ("starts-at")
    elif line.find("ends-at")!=-1:
        return ("ends-at")  

In [ ]:
def getArguments(line):
    return [ entry.strip() for entry in line.split(findOperation(line)) ]

In [ ]:
def getColumns():
    header = ["Row"] + [str(time) for time in range(8,17+1)]
    columns = dict( [column, None] for column in header )
    return columns

In [ ]:
def makeRows(commands):
    rows = []
    for line in commands:
        if findOperation(line)=="has-row":
            rows.append(getArguments(line)[1])
    return rows

In [ ]:
def getTime(commands):
    slots = []
    for line in commands:
        if findOperation(line)=="starts-at":
            slots.append(getArguments(line)[0]) 
    slots = dict([slot, None] for slot in slots)
    
    for slot in slots:
        related = [line for line in commands if getArguments(line)[0]==slot]
        start = [getArguments(line)[1] for line in related if findOperation(line)=="starts-at"]
        end = [getArguments(line)[1] for line in related if findOperation(line)=="ends-at"]
        start = int(start[0].split(':')[0])
        end = int(end[0].split(':')[0])

        slots[slot] = [str(time) for time in range(start, end+1)]    
    return slots

In [ ]:
def getSlots(commands):
    slots = dict([ [row, list()] for row in makeRows(commands) ])
    for line in commands:
        if findOperation(line)=="has-slot":
            row, slot = getArguments(line)
            slots[row].append([slot, getTime(commands)[slot]])
    return slots

In [ ]:
def makeTable():
    slots = getSlots(commands)
    rows = []
    for row in slots.keys():
        data = getColumns()
        data["Row"] = row
        for slot in slots[row]:
            for time in slot[1]:
                data[time] = slot[0]
        rows.append(data)
    return rows

In [ ]:
def canOverlap(table, row1, row2):
    flag=True
    for row in contents:
        if row["Row"]==row1:
            rowA = row
        if row["Row"]==row2:
            rowB = row

    for key in getColumns().keys():
        if key!="Row":
            if rowA[key]!=None and rowB[key]!=None:
                flag=False
                break

    return flag

In [ ]:
def getClashes(table):
    clashes = []
    pairable = []
    rows = makeRows(commands)
    for row1 in rows:
        for row2 in rows:
            if row1!=row2:
                if canOverlap(table, row1, row2):
                    if pairable.count(sorted([row1,row2]))==0:
                        pairable.append(sorted([row1,row2]))
                else:
                    if clashes.count(sorted([row1,row2]))==0:
                        clashes.append(sorted([row1,row2]))
    return clashes, pairable

In [ ]:
def rename(table):
    for i in range(table.shape[0]):
        table = table.rename(index={i: table.iloc[i]["Row"]})
    table.drop(columns=["Row"], inplace=True)
    return table

In [ ]:
contents = makeTable()

In [ ]:
timetable = pd.DataFrame(makeTable())

In [ ]:
timetable.style.hide_index()

In [ ]:
import dataframe_image as dfi

In [ ]:
df_styled = timetable.style.background_gradient()

In [ ]:
dfi.export(df_styled,"mytable.png")